In [ ]:
import sqlite3
import pandas as pd
from fuzzywuzzy import process
import numpy as np
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

C:\Users\rai\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Establish connection to database 

In [ ]:
db = sqlite3.connect('moviesrecommender1.db')

### Creation of tables

In [4]:
tables = """
CREATE TABLE IF NOT EXISTS movies (
    movieid INTEGER ,
    title TEXT NOT NULL,
    genres VARCHAR(32)
    
);
"""
db.executescript(tables)

In [5]:
tables = """
CREATE TABLE IF NOT EXISTS ratings (
    userId INTEGER,
    movieId INTEGER,
    rating INTEGER,
    timestamp INTEGER
);
"""
db.executescript(tables)

In [6]:
tables = """
CREATE TABLE IF NOT EXISTS links (
    movieid INTEGER,
    imdbid INTEGER,
    tmdbid INTEGER
);
"""
db.executescript(tables)

In [7]:
tables = """
CREATE TABLE IF NOT EXISTS tags (
    userId INTEGER,
    movieId INTEGER,
    tag TEXT,
    timestamp INTEGER
);
"""
db.executescript(tables)

### Load data from csv files into pandas dataframe

The csv files consists of
* Movies
* Ratings
* Tags
* Links

In [8]:
file = ['movies','ratings','links','tags']
def dataframe(file):
    df = pd.read_csv(file+'.csv')
    return df

### Append dataframes to the tables created in database

In [9]:
for i in file:
    dataframe(i).to_sql('{}'.format(i),con = sqlite3.connect('moviesrecommender1.db'),
                        if_exists='append', index=False)

### Select the desired the columns from database tables

In [10]:
dataframe = pd.read_sql("""SELECT 
  ratings.userId,
  ratings.movieId,
  ratings.rating,
  ratings.timestamp,
  movies.title,
  movies.genres,
  tags.tag,
  links.imdbId,
  links.tmdbId
    
    
FROM ratings 
INNER JOIN movies
ON ratings.movieId = movies.movieId
LEFT JOIN tags
ON ratings.movieId= tags.movieID AND ratings.userId = tags.userId
INNER JOIN links
ON ratings.movieId = links.movieId

""", db)

### Inspect the created dataframe

In [11]:
dataframe.shape

(2959983, 9)

### Building a recommender system based on user rating

In [12]:
df = dataframe.pivot_table(values = 'rating',index= 'userId', columns='title',aggfunc='first' )

In [13]:
df1 = dataframe.pivot_table(values = 'rating',index= 'userId', columns='title',aggfunc='first' ).fillna(3)

In [14]:
df1.head(2)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Movies to select based on user search word using fuzzywuzzy filtering

* Enter the search word for movies

In [15]:
def movieresult():
    movie1_searchword = input()
    a = process.extract(movie1_searchword, df.columns)
    list2 = [x[0] for x in a]
    return list2
   

In [16]:
movies_suggestion1 = movieresult()
movies_suggestion2 = movieresult()
movies_suggestion3 = movieresult()

hello
girl
boy


### List of suggested movies

In [17]:
movies_suggestion1, movies_suggestion2, movies_suggestion3

(['Hello Again (1987)',
  'Hello Ladies: The Movie (2014)',
  'Hello, Dolly! (1969)',
  'Oh, Hello: On Broadway (2017)',
  'Othello (1995)'],
 ['100 Girls (2000)',
  '4 Little Girls (1997)',
  'All the Real Girls (2003)',
  'Bad Girls (1994)',
  'Beautiful Girls (1996)'],
 ['About a Boy (2002)',
  'Baby Boy (2001)',
  'Bad Boy Bubby (1993)',
  'Bad Boys (1995)',
  'Bad Boys II (2003)'])

### List of selected movies

In [18]:
movies_to_rate = [movies_suggestion1[0],movies_suggestion2[2],movies_suggestion3[4]]
movies_to_rate 


['Hello Again (1987)', 'All the Real Girls (2003)', 'Bad Boys II (2003)']

### Rate the selected movies

In [19]:
def rating(i):
    Enter_rating = input()
    return int(Enter_rating)

In [20]:
rating_first = rating(movies_suggestion1[0])
rating_two = rating(movies_suggestion2[2])
rating_three = rating(movies_suggestion3[4])

2
4
5


In [21]:
movies_rating = [rating_first,rating_two,rating_three]
movies_rating

[2, 4, 5]

In [22]:
new = list(zip(movies_to_rate,movies_rating))
new

[('Hello Again (1987)', 2),
 ('All the Real Girls (2003)', 4),
 ('Bad Boys II (2003)', 5)]

### Dataframe for entering user ratings into dataframe

In [23]:
user_rating = pd.DataFrame(np.zeros(shape=(1,len(df.columns))),columns=df.columns)

In [24]:
for a,b in new:
    for i in user_rating:
        user_rating[a]= user_rating[a].replace(0,b)

In [25]:
user_rating1 = user_rating.replace(0,3)
user_rating2 = user_rating.replace(0,np.nan)

### Building NMF model

In [26]:
nmf = NMF(n_components=3)
nmf.fit(df1)
hidden_profile = nmf.transform(user_rating1)

### Reconstruct ratings for all movies

In [27]:
nmf.components_.shape

(3, 9719)

In [28]:
ypred = np.dot(hidden_profile, nmf.components_)
ypred

array([[2.99522689, 3.00061554, 3.00155866, ..., 3.00863156, 3.00064128,
        2.99754783]])

### Output the best ranking movies that user hasnt rated yet

In [29]:
recom = pd.DataFrame(ypred.T, index=df1.columns)

In [30]:
recom['profile'] = user_rating2.T
recom[recom['profile'].isna()].sort_values(by=0, ascending=False)[:5]

,0,profile
title,,
"Shawshank Redemption, The (1994)",3.522500,NaN
Forrest Gump (1994),3.428640,NaN
"Silence of the Lambs, The (1991)",3.284713,NaN
Pulp Fiction (1994),3.283414,NaN
Braveheart (1995),3.253286,NaN


### Clustering movies based on genres

In [31]:
genres = list(dataframe['genres'])
len(genres)

2959983

In [32]:
genres_split = []
for g in genres:
    sublist = g.split('|')
    genres_split.append(sublist)

In [33]:
genres_split[:5]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']]

In [34]:
list_genre = [' '.join(x) for x in genres_split]
len(list_genre)

2959983

### Sparse matrix for genres column

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_train = count_vectorizer.fit_transform(list_genre)
index = list(sorted(count_vectorizer.vocabulary_.keys()))
index[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [36]:
df_Total = pd.DataFrame(count_train.todense(), columns = index)
df_Total.shape

(2959983, 24)

In [37]:
data_cluster = pd.concat([dataframe['title'], df_Total], axis = 1)

In [38]:
data_cluster.head(2)

,title,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,Toy Story (1995),0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Toy Story (1995),0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [39]:
data_cluster.drop('title', axis=1,inplace=True)

In [40]:
train=data_cluster.sample(frac=0.8,random_state=200)
test=data_cluster.drop(train.index)

In [41]:
train.shape, test.shape

((2367986, 24), (591997, 24))

In [42]:
model = KMeans(n_clusters=3)
model.fit(test)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [43]:
labels = model.predict(test)

labels[:10]

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 1])

### Evaluating Cluster

In [47]:
ks = range(1, 15)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)
    
    # Fit model to samples
    model.fit(train)
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)
    
# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)

KeyboardInterrupt: 